In [6]:
#Chatbot Training Model

from spellchecker import SpellChecker
from textblob import TextBlob
import emoji
import time
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('updated_intents.json',encoding="utf8").read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\DELL\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:81: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


237 documents
48 classes ['Age_group', 'Cloud_features', 'Discounts', 'Experiences', 'Impact', 'Inner_explorer', 'Language', 'Login', 'Login_issue', 'Mindfulness', 'Mindfulness_Programs', 'Mobile_app', 'Name', 'Need_mindfulness', "Parent's-concern", 'Play_audio', 'Signup', "Teacher's-concern", 'Well_being', 'advicefor_child', 'all_cost', 'app_availability', 'appreciation', 'audio_language', 'bad', 'chatbot', 'cloud_cost', 'community_cost', 'demo', 'district_cost', 'download_app', 'goodbye', 'greeting', 'invites', 'no_access_student', 'noanswer', 'parent', 'practice-detail', 'program_advice', 'reviews-experiences', 'school district', 'schools', 'single_school', 'smart', 'steps', 'support', 'thanks', 'work']
256 unique lemmatized words ["'m", "'s", "'ve", ',', '.', '1', '13', '2', '3', 'a', 'about', 'academic', 'account', 'achievement', 'actually', 'additional', 'advantage', 'age-group', 'agenda', 'aggressive', 'all', 'am', 'amazing', 'an', 'and', 'answer', 'anxiety', 'any', 'anyone', 'a

48/48 [==============================] - 0s 715us/step - loss: 0.5293 - accuracy: 0.8252
Epoch 51/200
48/48 [==============================] - 0s 720us/step - loss: 0.5202 - accuracy: 0.8306
Epoch 52/200
48/48 [==============================] - 0s 728us/step - loss: 0.4859 - accuracy: 0.8252
Epoch 53/200
48/48 [==============================] - 0s 778us/step - loss: 0.5688 - accuracy: 0.8256
Epoch 54/200
48/48 [==============================] - 0s 730us/step - loss: 0.4619 - accuracy: 0.8462
Epoch 55/200
48/48 [==============================] - 0s 734us/step - loss: 0.4273 - accuracy: 0.8635
Epoch 56/200
48/48 [==============================] - 0s 783us/step - loss: 0.5456 - accuracy: 0.8272
Epoch 57/200
48/48 [==============================] - 0s 742us/step - loss: 0.4025 - accuracy: 0.8773
Epoch 58/200
48/48 [==============================] - 0s 805us/step - loss: 0.4698 - accuracy: 0.8684
Epoch 59/200
48/48 [==============================] - 0s 798us/step - loss: 0.4626 - accuracy: 

48/48 [==============================] - 0s 1ms/step - loss: 0.2880 - accuracy: 0.9113
Epoch 131/200
48/48 [==============================] - 0s 1ms/step - loss: 0.2334 - accuracy: 0.9110
Epoch 132/200
48/48 [==============================] - 0s 1ms/step - loss: 0.2159 - accuracy: 0.9127
Epoch 133/200
48/48 [==============================] - 0s 932us/step - loss: 0.2794 - accuracy: 0.9262
Epoch 134/200
48/48 [==============================] - 0s 1ms/step - loss: 0.2343 - accuracy: 0.9163
Epoch 135/200
48/48 [==============================] - 0s 990us/step - loss: 0.2390 - accuracy: 0.9464
Epoch 136/200
48/48 [==============================] - 0s 1ms/step - loss: 0.2357 - accuracy: 0.9023
Epoch 137/200
48/48 [==============================] - 0s 989us/step - loss: 0.2184 - accuracy: 0.9366
Epoch 138/200
48/48 [==============================] - 0s 1ms/step - loss: 0.2923 - accuracy: 0.9099
Epoch 139/200
48/48 [==============================] - 0s 955us/step - loss: 0.3247 - accuracy: 0.8

In [7]:
from spellchecker import SpellChecker
import random
import multiprocessing
from flask import Flask,json, render_template, request
from flask_cors import CORS
import time
import nltk
from multiprocessing import Process
from nltk.stem import WordNetLemmatizer
from tensorflow.python.keras.backend import reset_uids
lemmatizer = WordNetLemmatizer()
import pickle
import json
import datetime
import numpy as np
import re
import os
import pyttsx3
import time
import playsound
import speech_recognition as sr
from gtts import gTTS
from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('updated_intents.json',encoding="utf8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
currentTime = datetime.datetime.now()
currentTime.hour
# app= Flask(__name__)
# CORS(app)
# @app.route('/run') 
def greeting():
    if currentTime.hour < 12:
        return {"Bot": ['Hi,Good Morning! Genie at service.']}
    elif 12 <= currentTime.hour < 18:
        return {"Bot": ['Hi,Good Afternoon! Genie at service.']}
    else:
        return {"Bot": ['Hi,Good evening! Genie at service.']}
# @app.route('/run/<msg>') 
def chatbot(msg):
    msg=SpellChecker().correction(msg)
    
    def clean_up_sentence(sentence):
        # token,aize the pattern - split words into array
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words
    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def bow(sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0]*len(words)  
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
        return(np.array(bag))
    def predict_class(sentence, model):
        # filter out predictions below a threshold
        p = bow(sentence, words,show_details=False)
        
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list 
        print(return_list )
    
    def getResponse(ints, intents_json):
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                result = random.choice(i['responses'])
                #result1 = random.choice(i['responses1'])
                break
        return result
        
    
    def chatbot_response(text):
        ints = predict_class(text, model)
        res = getResponse(ints, intents)
        return res
    
    
#     engine = pyttsx3.init('nsss')
    engine=pyttsx3.init('sapi5')
    voices=engine.getProperty('voices')
#     voice_id = "com.apple.speech.synthesis.voice.samantha"
#     engine.setProperty('voice',voice_id)
    engine.setProperty('voice','voices[1].id')
#     RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    RE_EMOJI = re.compile(
    "(["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "])"
  )
  
    symbols = ["</li><li>","</a>","<ol><li>","href=>","<a","</li></ol>\n","/","li","ol"]

    def strip_emoji(text):
#         text = text.strip(symbols)
        for i in symbols :
            text = text.replace(i, '')
        return RE_EMOJI.sub(r'', text)


    def speak(text):
        engine.say(text)
        engine.runAndWait()
    def get_audio():
        r = sr.Recognizer()
        with sr.Microphone() as source:
            audio = r.listen(source)
            said = ""
            try:
                said = r.recognize_google(audio)
                print(said)
            except Exception as e:
                print("Exception: " + str(e))
        return said
    def send():
        print(msg,"input")
        while True:#it used to be in continunation and to stop it press alt+cntrl+c
            #msg=input("You:- ")
            
            if msg=="quit":
                break
            else:
                res=chatbot_response(msg)
                prob=predict_class(msg, model)
                res1 = strip_emoji(res)
            
                
                return {"Bot":[res],"probability":[prob]},speak(res1)
                
                
                # return {"Bot":[res]}
    return send()
    
        
#     if __name__ == '__main__':
#         p = Process(target=chatbot)
#         p.start()
#         p.join()
# if __name__== "__main__":
#     app.run(threaded=False,port=5000)
#     # app.run(host='172.31.52.129',port=5000)

In [8]:
chatbot("Hello")

Hello input


({'Bot': ["Hello, I'm Genie. I spend all my days assisting people who want to talk to Inner Explorer! What can I do for you?😊"],
  'probability': [[{'intent': 'greeting', 'probability': '0.99990547'}]]},
 None)

In [4]:
# ['No worries,I am right here to guide you and to clarify your all possible doubts.\nPlease provide some details for suggesting 
#  to you the best program.<ol><li>Are you parents?</li><li>Are you leading a single school?</li><li>Are you running school 
#  district?</li></ol>']

In [5]:
# # CODE TO GET CHATBOT TAGS AND QUESTION PATTERNS

# import random
# import json     
# import pandas as pd

# data_file = open('updated_intents.json',encoding="utf8").read()
# intents = json.loads(data_file)
# p = []
# t =[]
# for intent in intents['intents']:
#     t.append(intent['tag'])
#     p.append(intent['patterns'])
        
# res = {}
# for key in t:
#     for value in p:
#         res[key] = value
#         p.remove(value)
#         break  

# ques = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in res.items() ]))
# quest= ques.T.fillna("")
# quest.to_excel("chatbot_questions.xlsx")